## Sentiment Analysis with Logistic Regression

This dataset is from a real contest of Text Processing.

The task is to build a model that will determine the tone (positive, negative) of the text. To do this, you will need to train the model on the existing data (train.csv). The resulting model will have to determine the class (neutral, positive, negative) of new texts. The dataset contains the following fields:

| Field name | Meaning |
|------------|-----------|
| ItemID  | id of twit|
| Sentiment | sentiment (1-positive, 0-negative)|
| SentimentText | text of the twit|

Let's first of all have a look at the data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [2]:
sentiment = pd.read_csv('https://raw.githubusercontent.com/dhminh1024/practice_datasets/master/twitter_sentiment_analysis.csv', encoding='latin-1')
sentiment.sample(5)

,ItemID,Sentiment,SentimentText
55608,55620,0,@babblingbrookie it doesn't come on until fall...
41564,41576,0,@anasimtiaz @ammar_faheem Waise le lena chahiy...
51018,51030,0,@ameonna10 oh my chocolate!! he shall replace...
62048,62060,0,@BigRedinTejas I was not pleased we left in t...
1633,1634,1,Completo. Just as the bell rings


In [3]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99989 entries, 0 to 99988
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ItemID         99989 non-null  int64 
 1   Sentiment      99989 non-null  int64 
 2   SentimentText  99989 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


As we can see, the structure of a twit varies a lot between twit and twit. They have different lengths, letters, numbers, strange characters, etc. 

It is also important to note that **a lot** of words are not correctly spelled, for example, the word _"Juuuuuuuuuuuuuuuuussssst"_ or the word _"sooo"_

This makes it hard to measure how positive or negative the words are within the twits.

So we need a way of scoring the words such that words that appear in positive twits have a greater score than those that appear in negative twits.

But first, how do we represent the twits as vectors we can input to our algorithm?

### Bag of words

One thing we could do to represent the twits as equal-sized vectors of numbers is the following:

* Create a list (vocabulary) with all the unique words in the whole corpus of twits. 
* We construct a feature vector from each twit that contains the counts of how often each word occurs in the particular twit

_Note that since the unique words in each twit represent only a small subset of all the words in the bag-of-words vocabulary, the feature vectors will mostly consist of zeros_

Let's construct the bag of words. We will work with a smaller example for illustrative purposes, and in the end, we will work with our real data.

In [4]:
twits = [
    'This is amazing!',
    'ML is the best, yes it is',
    'I am not sure about how this is going to end...'
]

Let's import [CountVectorizer.](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) It'll help us to convert a collection of text documents to a matrix of token counts.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Define an object of CountVectorizer() fit and transfom your twits into a 'bag'
count = CountVectorizer()
bag = count.fit_transform(twits)

In [6]:
# Find in document of CountVectorizer a function that show us list of feature names
count.get_feature_names()

['about',
 'am',
 'amazing',
 'best',
 'end',
 'going',
 'how',
 'is',
 'it',
 'ml',
 'not',
 'sure',
 'the',
 'this',
 'to',
 'yes']

As we can see from executing the preceding command, the vocabulary is stored in a Python array that maps the unique words to integer indices. Next, let's print the feature vectors that we just created:

In [7]:
# Call toarray() on your 'bag' to see the feature vectors
bag.toarray()

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]])

**Quiz**: What is the index of the word 'is' and how many times it occurs in all three twits?

Each index position in the feature vectors corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. For example, the first feature at index position 0 resembles the count of the word 'about', which only occurs in the last document. These values in the feature vectors are also called the **raw term frequencies**: `tf(t,d )` —the number of times a term `t` occurs in a document `d`.

### How relevant are words? Term frequency-inverse document frequency

We could use these raw term frequencies to score the words in our algorithm. There is a problem though: If a word is widespread in _all_ documents, then it probably doesn't carry a lot of information. To tackle this problem, we can use **term frequency-inverse document frequency**, which will reduce the score, the more frequent the word is across all twits. It is calculated like this:

\begin{equation*}
tf-idf(t,d) = tf(t,d) ~ idf(t,d)
\end{equation*}

_tf(t,d)_ is the raw term frequency descrived above. _idf(t,d)_ is the inverse document frequency, than can be calculated as follows:

\begin{equation*}
\log \frac{n_d}{1+df\left(d,t\right)}
\end{equation*}

Where `n` is the total number of documents, and _df(t,d)_ is the number of documents where the term `t` appears. 

The `1` addition in the denominator is just to avoid zero terms for terms that appear in all documents, will not be entirely ignored. And the `log` ensures that low-frequency term doesn't get too much weight.

Fortunately for us, `scikit-learn` does all those calculations for us:

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
# Feed the tf-idf Vectorizer with twits using fit_transform()
tfidf_vec = tfidf.fit_transform(twits)

# Formatting the number to 2 digits after the decimal point by showing on this notebook
np.set_printoptions(precision=2)
# To print array in one line
np.set_printoptions(linewidth=np.inf)
print(tfidf.get_feature_names())
print(tfidf_vec.toarray())

['about', 'am', 'amazing', 'best', 'end', 'going', 'how', 'is', 'it', 'ml', 'not', 'sure', 'the', 'this', 'to', 'yes']
[[0.   0.   0.72 0.   0.   0.   0.   0.43 0.   0.   0.   0.   0.   0.55 0.   0.  ]
 [0.   0.   0.   0.4  0.   0.   0.   0.47 0.4  0.4  0.   0.   0.4  0.   0.   0.4 ]
 [0.33 0.33 0.   0.   0.33 0.33 0.33 0.2  0.   0.   0.33 0.33 0.   0.25 0.33 0.  ]]


## Data cleaning and manipulation

Let's work on our real vocabulary

### Removing stopwords

**Stop words** are the most common words are meaningless in terms of sentiment: I, to, the, and... they don't give any information on positiveness or negativeness. They're basically noise that can most probably be eliminated. Therefore, it is a common practice to remove them when doing text analysis.

In [9]:
from collections import Counter

vocab = Counter()
for twit in sentiment.SentimentText:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('', 123916),
 ('I', 32879),
 ('to', 28810),
 ('the', 28087),
 ('a', 21321),
 ('you', 21180),
 ('i', 15995),
 ('and', 14565),
 ('it', 12818),
 ('my', 12385),
 ('for', 12149),
 ('in', 11199),
 ('is', 11185),
 ('of', 10326),
 ('that', 9181),
 ('on', 9020),
 ('have', 8991),
 ('me', 8255),
 ('so', 7612),
 ('but', 7220)]

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop_words' 
# and save them in vocab_reduced
for w, c in vocab.items():
    if not w in stop_words:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[('', 123916),
 ('I', 32879),
 ("I'm", 6416),
 ('like', 5086),
 ('-', 4922),
 ('get', 4864),
 ('u', 4194),
 ('good', 3953),
 ('love', 3494),
 ('know', 3472),
 ('go', 2990),
 ('see', 2868),
 ('one', 2787),
 ('got', 2774),
 ('think', 2613),
 ('&amp;', 2556),
 ('lol', 2419),
 ('going', 2396),
 ('really', 2287),
 ('im', 2200)]

### Removing special characters

If you look closer, you'll see that we're also taking into consideration punctuation signs ('-', ',', etc) and other html tags like `&amp`. We can definitely remove them for the sentiment analysis, but we will try to keep the emoticons, since those _do_ have a sentiment load:

In [12]:
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

# Create some random texts for testing the function preprocessor()
print(preprocessor('I like it :), |||<><>'))

i like it  :)


### Stemming

We are almost ready! There is another trick we can use to reduce our vocabulary and consolidate words. If you think about it, words like love, loving, etc. _Could_ express the same positivity. If that was the case, we would be having two words in our vocabulary when we could have only one: lov. This process of reducing a word to its root is called **stemming**.

We also need a _tokenizer_ to break down our twits in individual words. We will implement two tokenizers, a regular one and one that does stemming:

In [13]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

# Split a text into list of words
def tokenizer(text):
    return text.split()

# Split a text into list of words and apply stemming technic
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# Testing
print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there,', 'I', 'am', 'loving', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there,', 'I', 'am', 'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


### Train Logistic Regression

In [14]:
from sklearn.model_selection import train_test_split

X = sentiment['SentimentText']
y = sentiment['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7f518166c488>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7f518166cbf8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.7525752575257526
confusion matrix:
 [[5828 2877]
 [2071 9222]]
classification report:
               precision    recall  f1-score   support

           0       0.74      0.67      0.70      8705
           1       0.76      0.82      0.79     11293

    accuracy                           0.75     19998
   macro avg       0.75      0.74      0.75     19998
weighted avg       0.75      0.75      0.75     19998



Finally, let's run some tests

In [17]:
twits = [
    "I do not feel not bad", # Phuc +1
    'This model is "so good" :))', # Long -1
    'we are who we are', # Nghi 0
    'its good to be bad sometimes', # PA +1
    'what a wonderful failure! (sarcasm :)))', #Phuc +1
    'People do not like the bad things', # Chi 0
    'We finally have the test result. You are positive', # Long +1
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

I do not feel not bad --> Negative, Positive = [0.99 0.01]
This model is "so good" :)) --> Negative, Positive = [0.16 0.84]
we are who we are --> Negative, Positive = [0.39 0.61]
its good to be bad sometimes --> Negative, Positive = [0.55 0.45]
what a wonderful failure! (sarcasm :))) --> Negative, Positive = [0.31 0.69]
People do not like the bad things --> Negative, Positive = [0.79 0.21]
We finally have the test result. You are positive --> Negative, Positive = [0.2 0.8]


If we would like to use the classifier in another place, or just not train it again and again everytime, we can save the model in a pickle file:

In [18]:
import pickle
import os

pickle.dump(clf, open('logisticRegression.pkl', 'wb'))

And reload it in your app:

In [19]:
with open('logisticRegression.pkl', 'rb') as model:
    reload_model = pickle.load(model)
preds = reload_model.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

I do not feel not bad --> Negative, Positive = [0.99 0.01]
This model is "so good" :)) --> Negative, Positive = [0.16 0.84]
we are who we are --> Negative, Positive = [0.39 0.61]
its good to be bad sometimes --> Negative, Positive = [0.55 0.45]
what a wonderful failure! (sarcasm :))) --> Negative, Positive = [0.31 0.69]
People do not like the bad things --> Negative, Positive = [0.79 0.21]
We finally have the test result. You are positive --> Negative, Positive = [0.2 0.8]


**Well done!**